In [2]:
%load_ext autoreload
%autoreload 2
import ipyleaflet as L
from dataloader import TripsLoader
from typing import Literal
from ipywidgets import Layout


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
loader = TripsLoader('03-07-2022')

In [4]:
def get_trips_by_load(load: Literal['Stone', 'Equipment', 'Soil', '4']):
    pass

In [5]:
#<a href="https://www.flaticon.com/free-icons/excavator" title="excavator icons">Excavator icons created by nawicon - Flaticon</a>
#<a href="https://www.flaticon.com/free-icons/dump" title="dump icons">Dump icons created by Freepik - Flaticon</a>
m = L.Map(layout=Layout(width='60%', height='700px'), center=[59.95, 10.3])
machine = loader._machines[36]
for trip in machine.trips[:100]:
    load_icon = L.Icon(icon_url='https://cdn-icons-png.flaticon.com/512/2716/2716797.png', icon_size=[32, 32], icon_anchor=[16,16])
    load_mark = L.Marker(location=trip.load_latlon, icon=load_icon, rotation_angle=0, rotation_origin='22px 94px')
    m.add_layer(load_mark)
    dump_icon = L.Icon(icon_url='https://cdn-icons-png.flaticon.com/512/1435/1435320.png', icon_size=[32, 32], icon_anchor=[16,16])
    dump_mark = L.Marker(location=trip.dump_latlon, icon=dump_icon, rotation_angle=0, rotation_origin='22px 94px')
    m.add_layer(dump_mark)
display(m)

Map(center=[59.95, 10.3], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_ou…

In [7]:
import rasterio as rio
from pyproj import Transformer
import cv2

for i, place in enumerate(["Skaret", "Nordlandsdalen"]):
    in_path = f'data/DroneData_2022-03-18_to_2022-05-22/SINTEF - Bjørumskaret/P0{i + 7}_22.03.29_{place}-Orthomosaic.tiff'

    with rio.open(in_path) as src:
        img = src.read()
        min_lon, min_lat, max_lon, max_lat = src.bounds

    transformer = Transformer.from_crs("EPSG:5110", "EPSG:4326")
    cv2.imwrite(f'../tmp/{place}.png', cv2.cvtColor(img.transpose(1, 2, 0), cv2.COLOR_RGB2BGR))
        
    ## Conversion from UTM to WGS84 CRS
    bounds_orig = [transformer.transform(*bounds) for bounds in [[min_lat, min_lon], [max_lat, max_lon]]]
    print(bounds_orig)
    # Overlay raster (RGB) called img using add_child() function (opacity and bounding box set)
    img= L.ImageOverlay(url=f'../tmp/{place}.png',
                                    bounds = bounds_orig)
    m.add_layer(img)
m

ImportError: libGL.so.1: cannot open shared object file: No such file or directory